### Run notebook 01_LoL_predictor_EDA_preparation before running this one, as it prepares the data!

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import yaml

In [2]:
with open('config.yml', mode="r") as f:
    config = yaml.safe_load(f)

import wandb
wandb.login()

wandb.init(
        project="lol-predictor",
        entity="adamzak",
        config=config['train'],
        group='pytorch',
        mode='online'
    )

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: ERROR Unable to read ~/.netrc
wandb: Currently logged in as: adamzak. Use `wandb login --relogin` to force relogin


wandb: ERROR Failed to sample metric: Not Supported


In [3]:
pd.set_option('display.max_columns', 500)
PREPROCESSED_PATH = '../data/processed_high_diamond.csv'

data = pd.read_csv(PREPROCESSED_PATH)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

wandb: ERROR Failed to serialize metric: division by zero


In [4]:
data.drop(columns={"gameId"}, inplace=True)

In [5]:
# split data to train and test by 70/30 ratio
train, test = train_test_split(data, test_size=0.3)

# further split test part to 50/50 with validation - final split will be 70/15/15
test, validation = train_test_split(test, test_size=0.5)

In [6]:
train.shape

(6535, 17)

In [7]:
test.shape

(1401, 17)

In [8]:
validation.shape

(1401, 17)

In [9]:
trainX = train.drop(columns={"blueWins"})
trainy = train.blueWins

In [10]:
testX = test.drop(columns={"blueWins"})
testy = test.blueWins

In [11]:
nHidden = config['train']['inputNeurons']

model = nn.Sequential(
    nn.Linear(16, nHidden[0]),
    nn.ReLU(),
    nn.Linear(nHidden[0], nHidden[1]),
    nn.ReLU(),
    nn.Linear(nHidden[1], nHidden[2]),
    nn.ReLU(),
    nn.Linear(nHidden[2], nHidden[3]),
    nn.ReLU(),
    nn.Linear(nHidden[3], 1),
    nn.Sigmoid()
    # Softmax
)

In [12]:
trainX = torch.tensor(trainX.values)
testX = torch.tensor(testX.values)

In [13]:
trainy = torch.tensor(trainy.values)
testy = torch.tensor(testy.values)

# https://stackoverflow.com/a/60440460/12342419
trainy = trainy.type(torch.LongTensor)
testy = testy.type(torch.LongTensor)

In [14]:
def compute_loss(y_hat, y):
    return nn.BCELoss()(y_hat, y)

In [15]:
# Calculate accuracy (a classification metric)
def accuracy_fn(y_true, y_pred):
    correct = torch.eq(y_true, y_pred).sum().item() # torch.eq() calculates where two tensors are equal
    acc = (correct / len(y_pred)) * 100
    return acc

In [16]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [17]:
trainX = trainX.to(torch.float32).to(device)
testX = testX.to(torch.float32).to(device)
trainy = trainy.to(torch.float32).to(device)
testy = testy.to(torch.float32).to(device)

In [18]:
trainy = trainy.reshape((trainy.shape[0], 1))

In [19]:
model

Sequential(
  (0): Linear(in_features=16, out_features=256, bias=True)
  (1): ReLU()
  (2): Linear(in_features=256, out_features=128, bias=True)
  (3): ReLU()
  (4): Linear(in_features=128, out_features=64, bias=True)
  (5): ReLU()
  (6): Linear(in_features=64, out_features=32, bias=True)
  (7): ReLU()
  (8): Linear(in_features=32, out_features=1, bias=True)
  (9): Sigmoid()
)

In [20]:
model.to(device)
print(device)

cuda:0


In [21]:
torch.cuda.is_available()

True

In [22]:
torch.cuda.is_available()
torch.cuda.get_device_name(0)

'NVIDIA GeForce GTX 1050'

In [23]:
num_epochs = config['train']['epochs']

for n in range(num_epochs):
    model.train()
    y_pred = model(trainX)
    loss = compute_loss(y_pred, trainy)
    
    if n % (num_epochs / 100) == 0:
        print(n)
        #print(y_pred)
        print(accuracy_fn(y_true=trainy, y_pred=torch.round(y_pred)))
        accuracy = accuracy_fn(y_true=trainy, y_pred=torch.round(y_pred))
        wandb.log({'epoch': n, 'accuracy': accuracy, 'loss': loss})

    optimizer.zero_grad()
    loss.backward()
    #loss.backward(retain_graph=False)
    optimizer.step()
print(loss)

0
50.374904361132366
1000
76.81713848508033
2000
79.66335118592195
3000
78.71461361897475
4000
81.49961744452946
5000
83.02983932670237
6000
84.10099464422342
7000
84.10099464422342
8000
85.75363427697016
9000
79.34200459066565
10000
88.61514919663351
11000
87.28385615914308
12000
88.01836266258609
13000
90.83397092578423
14000
90.87987758224942
15000
92.19586840091813
16000
86.85539403213465
17000
94.123947972456
18000
94.95026778882938
19000
95.30221882172914
20000
95.31752104055087
21000
94.98087222647284
22000
97.04667176740628
23000
97.38332058148431
24000
96.40397857689365
25000
97.13848508033665
26000
98.74521805661821
27000
98.8982402448355
28000
99.12777352716144
29000
99.32670237184392
30000
99.02065799540934
31000
70.3442999234889
32000
98.50038255547055
33000
99.60214231063505
34000
99.80107115531752
35000
99.31140015302219
36000
95.24100994644223
37000
99.40321346595256
38000
99.98469778117827
39000
100.0
40000
100.0
41000
100.0
42000
100.0
43000
100.0
44000
100.0
45000
10

In [24]:
y_pred.shape

torch.Size([6535, 1])

In [25]:
torch.round(y_pred[:10])

tensor([[0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.]], device='cuda:0', grad_fn=<RoundBackward0>)

In [26]:
testy[:10]

tensor([0., 0., 1., 0., 0., 1., 0., 1., 1., 0.], device='cuda:0')

In [27]:
def predict(model, test_loader):
    all_preds = []
    all_preds_raw = []
    all_labels = []

    for batch in test_loader:
        batch.x = torch.tensor(batch.x)
        batch.x = batch.x.reshape((-1, *batch.x.shape[2:]))
        batch.to(device)  
        pred = model(torch.tensor(batch.x).float(), 
                        #batch.edge_attr.float(),
                        batch.edge_index, 
                        batch.batch) 

        all_preds.append(np.argmax(pred.cpu().detach().numpy(), axis=1))
        all_preds_raw.append(torch.sigmoid(pred).cpu().detach().numpy())
        all_labels.append(batch.y.cpu().detach().numpy())
    
    all_preds = np.concatenate(all_preds).ravel()
    all_labels = np.concatenate(all_labels).ravel()
    return all_preds, all_preds_raw, all_labels

In [28]:
torch.round(testX[0])

tensor([0., 1., 0., 0., 1., 1., 1., 1., 0., 0., 0., 0., 1., 1., 0., 1.],
       device='cuda:0')

In [29]:
testy

tensor([0., 0., 1.,  ..., 0., 0., 0.], device='cuda:0')

In [30]:
len(model(testX))

1401

In [31]:
model(testX)[0:10]

tensor([[1.0000e+00],
        [2.3166e-24],
        [4.9675e-01],
        [4.9675e-01],
        [4.9675e-01],
        [1.2295e-01],
        [4.8485e-01],
        [1.6043e-15],
        [5.0273e-01],
        [4.9675e-01]], device='cuda:0', grad_fn=<SliceBackward0>)

In [32]:
testy[:10]

tensor([0., 0., 1., 0., 0., 1., 0., 1., 1., 0.], device='cuda:0')

In [33]:
lol = model(testX[20:30])
print(lol)
#print(lol.flatten())
print(testy[20:30])

tensor([[3.8092e-06],
        [1.0000e+00],
        [1.5391e-02],
        [3.0884e-01],
        [4.9675e-01],
        [1.5088e-01],
        [4.9675e-01],
        [9.9898e-01],
        [1.0000e+00],
        [0.0000e+00]], device='cuda:0', grad_fn=<SigmoidBackward0>)
tensor([1., 1., 1., 0., 0., 0., 0., 0., 1., 1.], device='cuda:0')


In [34]:
testy[:10]

tensor([0., 0., 1., 0., 0., 1., 0., 1., 1., 0.], device='cuda:0')

In [35]:
predict(model, test)

AttributeError: 'str' object has no attribute 'x'

## Tensorflow implementation

In [ ]:
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt
import seaborn as sns
import tempfile
import os

In [37]:
import tensorflow as tf
from tensorflow import keras

In [38]:
nHidden = config['train']['inputNeurons']

tf_model = keras.Sequential([
    keras.layers.Dense(units=16, activation='relu'),
    keras.layers.Dense(units=nHidden[0], activation='relu'),
    keras.layers.Dense(units=nHidden[1], activation='relu'),
    keras.layers.Dense(units=nHidden[2], activation='relu'),
    keras.layers.Dense(units=nHidden[3], activation='sigmoid')  #activation='softmax'
])

2023-03-28 04:58:59.763555: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-03-28 04:58:59.789629: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [39]:
tf_model.compile(optimizer='adam', 
              loss=tf.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [43]:
history = tf_model.fit(trainX, trainy, epochs=10)

TypeError: Cannot convert the argument `type_value`: torch.float32 to a TensorFlow DType.